In [1]:
import numpy as np
import pandas as pd
import requests
import zipfile
import io

#  What hard-disk model shall we look at?

### The SMART metrics present in the dataset are model-dependent, thus the various models have to be treated differently. In this project we will focus on a single model. 

With this script we aim to capture the most convenient model to focus on. We deduce that the model ST4000DM000 is the most used throughout 2015, 2016, and 2017 with 37600 differend hard disks of which about 7% have failed.   

### We load the datasets and extract the relevant columns and save them in ModelsDetail_15, ModelsDetail_16, and ModelsDetail_17. The relevant columns are :

date : The time stamp of the observations.

serial_number : Uniquely identifies a hard-disk. It is used to determine the number of distinct entries.

model : Identifies the model of the hard-disk. 

failure : A value in {0,1}. When a 1 is present, the hard disk has failed on the specific date and is removed from the dataset.


In [2]:
def ReadFiles_EntriesFailure( url, start_string = ''):
    '''This function reads the zip file located at url, open it, read the columns needed to capture the ratio failures/entries'''
    ListDF = []
    
    r = requests.get(url)
    z = zipfile.ZipFile(io.BytesIO(r.content))

    files = [name for name in z.namelist() if (name.endswith('.csv')) & (name.startswith(start_string))]

    for file in files : 
        data = pd.read_csv(z.open(file))
        data2 = data[['date','serial_number', 'model', 'failure']]
        ListDF.append(data2)
    
    df = pd.concat(ListDF, ignore_index = True)
    
    return df
    
    
    
def ExtractEntriesAndFailures (df) :   
    '''Return dataframe with number of failures and unique serial numbers'''
    FailuresPerModel = df.groupby('model')['failure'].sum()
    EntriesPerModel = df.groupby('model')['serial_number'].unique()

    ModelsDetail = pd.concat([EntriesPerModel,FailuresPerModel],axis=1)
    return ModelsDetail
    

In [9]:
# String needed to access the data in the zipfiles stored at https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data

url_15 = 'https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_2015.zip'
start_str_15 = '2015/'

url_16_q1 = 'https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_Q1_2016.zip'
start_str_16_q1 = 'data_Q1_2016/'

url_16_q2 = 'https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_Q2_2016.zip'
start_str_16_q2 = 'data_Q2_2016/'

url_16_q3 = 'https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_Q3_2016.zip'
start_str_16_q3 = 'data_Q3_2016/'

url_16_q4 = 'https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_Q4_2016.zip'

url_17_q1 = 'https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_Q1_2017.zip'

url_17_q2 = 'https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_Q2_2017.zip'

url_17_q3 = 'https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_Q3_2017.zip'

url_17_q4 = 'https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_Q4_2017.zip'
start_str_17_q4 = 'data_Q4_2017/'

In [3]:
#Load data from 2015

ModelsDetail_15 = ExtractEntriesAndFailures(ReadFiles_EntriesFailure(url_15, start_str_15))

In [4]:
# Load data from 2016

df_16_q1 = ReadFiles_EntriesFailure(url_16_q1, start_str_16_q1)

df_16_q2 = ReadFiles_EntriesFailure(url_16_q2, start_str_16_q2)

df_16_q3 = ReadFiles_EntriesFailure(url_16_q3, start_str_16_q3)

df_16_q4 = ReadFiles_EntriesFailure(url_16_q4)

# Concatenate datasets of the 4 quartiles
df = pd.concat([df_16_q1, df_16_q2, df_16_q3, df_16_q4], ignore_index = True)

ModelsDetail_16 = ExtractEntriesAndFailures(df)

In [5]:
# Load data from 2017

df_17_q1 = ReadFiles_EntriesFailure(url_17_q1)

df_17_q2 = ReadFiles_EntriesFailure(url_17_q2)

df_17_q3 = ReadFiles_EntriesFailure(url_17_q3)

df_17_q4 = ReadFiles_EntriesFailure(url_17_q4, start_str_17_q4)

# Concatenate datasets of the 4 quartiles
df = pd.concat([df_17_q1, df_17_q2, df_17_q3, df_17_q4], ignore_index = True)

ModelsDetail_17 = ExtractEntriesAndFailures(df)

### For every year we saved the number of failures and an array of unique serial numbers

We combine the information across the three years in a common dataset. Observe that the same hard disk may have been used across different years.

In [7]:
ListDF_raw = [ModelsDetail_15.add_suffix('_15'), ModelsDetail_16.add_suffix('_16'), ModelsDetail_17.add_suffix('_17')]

#df_raw = reduce(lambda x, y: pd.merge(x, y, on = 'model'), ListDF_raw)

DfTot = pd.concat(ListDF_raw, axis = 1) 

DfTot.failure_15 = DfTot.failure_15.fillna(0) 
DfTot.failure_16 = DfTot.failure_16.fillna(0) 
DfTot.failure_17 = DfTot.failure_17.fillna(0) 

DfTot['failure_tot'] = DfTot.failure_15 + DfTot.failure_16 + DfTot.failure_17
del DfTot['failure_15']
del DfTot['failure_16']
del DfTot['failure_17']

DfTot['serial_number_tot'] = [np.hstack((DfTot.loc[mod,'serial_number_15'], DfTot.loc[mod,'serial_number_16'], DfTot.loc[mod,'serial_number_17'])) for mod in DfTot.index]

del DfTot['serial_number_15']
del DfTot['serial_number_16']
del DfTot['serial_number_17']

DfTot['entries_tot'] = [len(set(DfTot.loc[mod,'serial_number_tot'])) for mod in DfTot.index]

del DfTot['serial_number_tot']


For every model we have the number of distinct hard disks and the number of failures observed. 
We look at their ratio and filter the models for which too few hard disks have been sampled.

In [8]:
DfTot['ratio_entry_failures'] = DfTot['failure_tot'].divide(DfTot['entries_tot'], axis = 'rows')
DfTot.sort_values('ratio_entry_failures', inplace=True, ascending =False)

DfTot[DfTot['entries_tot'] > 1000]

,failure_tot,entries_tot,ratio_entry_failures
WDC WD30EFRX,122.0,1261,0.096749
ST3000DM001,106.0,1170,0.090598
ST4000DM000,2590.0,36700,0.070572
ST31500541AS,112.0,1693,0.066155
Hitachi HDS723030ALA640,44.0,1018,0.043222
Hitachi HDS722020ALA330,152.0,4683,0.032458
ST6000DX000,60.0,1938,0.030960
Hitachi HDS5C3030ALA630,96.0,4608,0.020833
Hitachi HDS5C4040ALE630,42.0,2660,0.015789
ST8000DM002,141.0,10029,0.014059


### Each brand has a different way of measuring the SMART metrics. For every model we need to understand which metrics are important and which are not. 

We now select a model and look at the data in the 2015 time frame. This is needed to obtain a mask that will be used to extract only the columns relevant to a specific model. 

Every model is associated with different SMART metrics. Moreover, for every relevant SMART metric we will work with the normalized value (in between 0 and 100/200/260 depending on the metric, the higher the better in tems of performance)

In [6]:
## The following functions are needed in order to build a mask for any specific model.

# (1) Read the datase associated to a model and a specific mask.
def ReadFiles_Model_and_RelCols (url, mod_name, rel_cols, start_string = ''):
    ''' This function reads the zip file located at url, open it, read the columns associated to a model and 
    the relevant columns '''
    ListDF = []
    r = requests.get(url)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    files = [name for name in z.namelist() if (name.endswith('.csv')) & (name.startswith(start_string))]
    for file in files : 
        data = pd.read_csv(z.open(file), parse_dates = [0])
        data2 = data[data['model'] == mod_name]
        data3 = data2.iloc[:,rel_cols]
        ListDF.append(data3)
    
    df = pd.concat(ListDF, ignore_index = True)
    return df

# (2) Return the SMART metrics with no NaN values and relevant variance.
def RelevantCols(df, thresh = 20) :
    ''' For the various SMART metrics return True only if the metric is considered by a specific model, i.e. the 
    relative values are not NaN and if the metric is relevant, i.e., the normalized range (max - min) is larger 
    than thresh'''
    non_na_cols = (df.isnull().sum() == 0)
        
    res_bool = []
        
    for col in df.columns :
        if non_na_cols[col] == True :
            diff = df[col].max() - df[col].min()
            if diff > thresh :
                res_bool = res_bool + [True]
            else :
                res_bool = res_bool + [False]
        else :
            res_bool = res_bool + [False]
    return res_bool
    
# (3) Build the mask given the result of RelevantCols    
def BuildTheMask(rel_cols) :
    ''' Esclude all raw columns and all the normalized columns not relevant for a specific model '''
    
    rel_cols_double = [val for val in rel_cols for _ in (0, 1)] 
    
    return [a and b for a, b in zip(rel_cols_double, normalized_cols)]    

In [3]:
# Select the model you want to work with

ModelName = 'Hitachi HDS722020ALA330' # (ex:'Hitachi HDS722020ALA330')

## normalized_cols has 'False' in correspondence of a column with a raw value and 'True' of a col with a normalized value

normalized_cols = [
                True,  False, True,  False, True,  False,  True,  False, # SMART 1 - 2 - 3 - 4 
                True,  False, True,  False, True,  False,  True,  False, # SMART 5 - 7 - 8 - 9 
                True,  False, True,  False, True,  False,  True,  False, # SMART 10 - 11 - 12 -13
                True,  False, True,  False, True,  False,  True,  False, # SMART 15 - 22 - 183 - 184 
                True,  False, True,  False, True,  False,  True,  False, # SMART 187 - 188 - 189 - 190
                True,  False, True,  False, True,  False,  True,  False, # SMART 191 - 192 - 193 - 194 
                True,  False, True,  False, True,  False,  True,  False, # SMART 195 - 196 - 197 - 198 
                True,  False, True,  False, True,  False,  True,  False, # SMART 199 - 200 - 201 - 220
                True,  False, True,  False, True,  False,  True,  False, # SMART 222 - 223 - 224 - 225
                True,  False, True,  False, True,  False,  True,  False, # SMART 226 - 240 - 241 - 242
                True,  False, True,  False, True,  False,  True,  False, True,  False] # SMART 250 - 251 - 252 - 254 - 255

# [date, serial number, model, capacity, failure] + SMART metrics
cols = [False, False, False, False, False] + normalized_cols 


In [4]:
# Call (1) with the 2015 dataset and keeping all the normalized columns
url_15 = 'https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_2015.zip'
start_str_15 = '2015/'
df15 = ReadFiles_Model_and_RelCols(url_15, ModelName, cols, start_string = start_str_15)

In [7]:
# Call (2) with the dataset df15 obtained with the previous call
relevant_cols = RelevantCols(df15) 

# Call (3) with the relevant columns relevant_cols    
mask = BuildTheMask(relevant_cols)    

### We have the mask relative to the model indicated in ModelName (ex:'Hitachi HDS722020ALA330'). 

We access the online data once again and save only the rows related to ModelName and the columns specified by the mask

In [10]:
# [date, serial number, model, capacity, failure] + SMART metrics
cols = [True, True, False, False, True] + normalized_cols 

# Path to the folder in which the datset will be stored
folder_name_model = ModelName.replace(' ', '_')
folder_name = 'Data/' + folder_name_model

In [12]:
# Save data of 2015

df_15 = ReadFiles_Model_and_RelCols(url_15, ModelName, cols, start_string = start_str_15)
df_15.to_csv(folder_name + '/Model_15.csv')

In [13]:
# Save data of 2016

df_16_q1 = ReadFiles_Model_and_RelCols(url_16_q1, ModelName, cols, start_str_16_q1)

df_16_q2 = ReadFiles_Model_and_RelCols(url_16_q2, ModelName, cols, start_str_16_q2)

df_16_q3 = ReadFiles_Model_and_RelCols(url_16_q3, ModelName, cols, start_str_16_q3)

df_16_q4 = ReadFiles_Model_and_RelCols(url_16_q4, ModelName, cols)

# Concatenate datasets of the 4 quartiles
df_16 = pd.concat([df_16_q1, df_16_q2, df_16_q3, df_16_q4], ignore_index = True)

df_16.to_csv(folder_name + '/Model_16.csv')

In [14]:
# Save data of 2017

df_17_q1 = ReadFiles_Model_and_RelCols(url_17_q1, ModelName, cols)

df_17_q2 = ReadFiles_Model_and_RelCols(url_17_q2, ModelName, cols)

df_17_q3 = ReadFiles_Model_and_RelCols(url_17_q3, ModelName, cols)

df_17_q4 = ReadFiles_Model_and_RelCols(url_17_q4, ModelName, cols, start_str_17_q4)

# Concatenate datasets of the 4 quartiles
df_17 = pd.concat([df_17_q1, df_17_q2, df_17_q3, df_17_q4], ignore_index = True)

df_17.to_csv(folder_name + '/Model_17.csv')